In [1]:
### TODO
#check if alignment works if tip is vertically at the top

#what happends if phasemaps not even?

#check all cells have user-editable inputs at the top

## Notebook for aligning projections

### 0. Global parameter definitions
### 1. Measuring miss-tilt and finding rotation axis
### 2. Creating 3-D mask

In [2]:
%matplotlib qt

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pickle

import skimage.filters as skfl

In [4]:
import fpd
import hyperspy.api as hs

In [5]:
#load editable pyramid version
import sys
import os
pyramid_dev_version_path=os.path.abspath(r"D:\Programmes\Git\empyre-patched\empyre")
sys.path.append(pyramid_dev_version_path)
#print(sys.path)
import pyramid as pr
pr

<module 'pyramid' from 'D:\\Programmes\\Git\\empyre-patched\\empyre\\pyramid\\__init__.py'>

In [6]:
#Code by A.S.
import pyramidas.alignment as pa
import pyramidas.util as pu
import pyramidas.reconstruction as pre

### 0. Global parameter definitions

In [7]:
#file paths

folders = ['+0',
 '+20',
 '+30',
 '-10',
 '-20',
 '-30',
 '-40',
 '-50',
 '-60']

# file names for single (phasemap + projector + mask) datasets
f_name="data.pickle"

In [8]:
#load all the phase images
no_projections=len(folders)
datas=[None]*no_projections

for i in range(len(datas)):
    data_path=folders[i]+'\\'+f_name
    with open(data_path, 'rb') as f:
        datas[i]=pickle.load(f)

In [9]:
#plot masks and phasemaps
masks=[]
phasemaps_raw=[]
for i in range(len(datas)):
    datas[i].plot_phasemaps()
    phasemap=datas[i].phasemaps[0]
    phasemaps_raw.append(phasemap)
    print(f"phasemap {i} shape", phasemap.dim_uv)
    
phasemaps=pa.pad_equalise_tilt_series(phasemaps_raw)
    
for pm in phasemaps:
    mask=pm.mask
    masks.append(mask)

phasemap 0 shape (120, 116)
phasemap 1 shape (120, 116)
phasemap 2 shape (120, 116)
phasemap 3 shape (120, 116)
phasemap 4 shape (120, 116)
phasemap 5 shape (120, 116)
phasemap 6 shape (120, 116)
phasemap 7 shape (120, 116)
phasemap 8 shape (120, 116)
Pixel spacing is 10.249 +/- 5.99e-07


In [20]:
#### for plotting later

ms=tuple(sorted(zip(folders,masks), key=lambda x: int(x[0])))
ms
names=[]
folder=".//masks 0//"
for el in ms:
    m=el[1].astype(int)*2
    m0=np.ones(m.shape)*1
    bw=3
    m0[bw:-bw,bw:-bw]=m[bw:-bw,bw:-bw] # add border
    m=(m0*255/np.max(m0)).astype('uint8')
    name=el[0]+".z0"
    names.append(str(name))
    plt.matshow(m)
    img=Image.fromarray(m)
    img.save(folder+name+".png")
print(names)

['-60.z0', '-50.z0', '-40.z0', '-30.z0', '-20.z0', '-10.z0', '+0.z0', '+20.z0', '+30.z0']


### 1. Measuring miss-tilt and finding rotation axis

In [10]:
#Before measuring the projection angles, shift all phase images such that the sample is in approximately the same position.

# method is "phase_correlation" or "centre_of_mass"
method="phase_correlation"
# index of the reference image all others should be compared to and shifted to match
test_image_index=0


shifts = pa.find_image_shifts(masks, test_image_index=test_image_index, method=method)
phasemaps_similar = pa.pad_translate_tilt_series(phasemaps, shifts)
projectors=[data.projectors[0] for data in datas]

In [11]:
#fit a line to each mask and determine wire angle. Also extract x-tilt values from projectors.

crop_right=85
crop_left=30
crop_bottom=80
crop_top=0

# crop_right=80
# crop_left=38
# crop_bottom=0
# crop_top=60


tana, xtilts = pa.measure_wire_orientations(phasemaps_similar, projectors, crop_right=crop_right, crop_left=crop_left, 
                                         crop_bottom=crop_bottom, crop_top=crop_top, plot_results=True, verbose=False)

In [119]:
# define an error function that goes to 0 if the correct axis of rotation is identified 
# Method is true for an arbitrarily orientated volume of revolution (e.g. cylinders and cones).
#methods = 'L-BFGS-B' or 'TNC'
# lowest 'distance from 0' is best

tilt_axis_direction_guess=np.radians(15)
miss_tilt_guess=np.radians(0)

sol, fun = pa.minimise_total_error(tana, xtilts, x0 = [tilt_axis_direction_guess, miss_tilt_guess], method='TNC', 
                                   error_window_width=np.radians(3)) 
th0, p0 = sol.x
#print(sol)

root theta: 9.51 deg +6.399 / -6.399
root x_tilt: 6.83 deg +1.324 / -1.324
distance from 0 error: 0.004591546880901653
     fun: 0.004591546880901653
     jac: array([ 5.89805982e-09, -1.41379963e-08])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 39
     nit: 10
  status: 1
 success: True
       x: array([0.16590235, 0.11927609])


In [86]:
#inspect the error function
pa.plot_3D_surface(fun)

#zoom in onto the solution in the error function
range_rad=np.radians(3)

bracket = [[-range_rad+th0, range_rad+th0],[-range_rad+p0, range_rad+p0]]
pa.plot_3D_surface(fun, bracket=bracket, title="Error function (zoomed in)")

In [120]:
#refine tilt and direction measurements
#rotates the images to have tilt axis be horizontal and accounts for miss-tilt

crop_right=77
crop_left=38
crop_bottom=80
crop_top=0

# crop_right=75
# crop_left=28
# crop_bottom=0
# crop_top=60

tana_ref, xtilts_ref = pa.measure_wire_orientations(phasemaps_similar, projectors, theta=th0, xtilt0=p0, 
                                                    crop_right=crop_right, crop_left=crop_left, 
                                         crop_bottom=crop_bottom, crop_top=crop_top, plot_results=True, verbose=False)


In [124]:
#align the phasemaps
tip_axis=1 # 0 -> y_axis, 1 -> x_axis #which direction should the wire tip stands out.
use_high_end=True
camera_rotation=0

#adjust to select the wanted feature on the flattest lying image
crop_right=70
crop_left=35
crop_bottom=60
crop_top=0



tilts = np.degrees(xtilts_ref)
axis_rot = np.degrees(th0)
phasemap_titles=["projection at %.1f deg"%tilt for tilt in tilts]

#rotate the phasemaps such that x-axis coresponds to tilt axis
print("\nrotating phasemaps")
phasemaps_rot=pa.rotate_phasemaps(phasemaps_similar, axis_rot, smooth_masks=True)
 

#centre the phase images such that the confidence region centres coincide
print("\ncentring phasemaps") 
phasemaps_centered = pa.centre_phasemaps(phasemaps_rot, padded=False)


#measure translations of the needle tip along the x-axis and correct
print("\nx-shifting phasemaps")
phasemaps_translated = pa.align_wire_tips(phasemaps_centered, axis=tip_axis, use_high_end=use_high_end, padded=False, verbose=True)

#trim unnecessary empty space
print("\ntrimming some empty space from phasemaps")
phasemaps_trim = pa.trim_empty_space(phasemaps_translated, equal_trim=True)

#calculate projections of a starting image and shift all others to match.
print("\naligning phasemap projections")
phasemaps_corrected, reconstruction_dimensions = pa.align_wire_directions(phasemaps_trim, tilts, plot_fits=True, plot_aligned_masks=True, crop_right=crop_right,
                        crop_left=crop_left, crop_top=crop_top, crop_bottom=crop_bottom,
                        test_mask_index=None, use_round_projection=False, 
                        axis=1, z_ang=0, camera_rotation=camera_rotation, subcount=5, padded=False, verbose=True)


#trim unnecessary empty space
print("\ntrimming all unnecessary empty space from phasemaps")
phasemaps_aligned = pa.trim_empty_space(phasemaps_corrected, equal_trim=False, verbose=False)

print("\nfinished alignments")


rotating phasemaps

centring phasemaps

x-shifting phasemaps
shifts (index, (dy,dx)): [(0, (0, -3)), (1, (0, 4)), (2, (0, 5)), (3, (0, 0)), (4, (0, -1)), (5, (0, 1)), (6, (0, -3)), (7, (0, -1)), (8, (0, -4))]

trimming some empty space from phasemaps

aligning phasemap projections
0 tilt mask is calculated from mask 3
Starting projector calculation for tilt 6.83 deg
Starting projector calculation for tilt 26.83 deg
Starting projector calculation for tilt 36.83 deg
Starting projector calculation for tilt -3.17 deg
Starting projector calculation for tilt -13.17 deg
Starting projector calculation for tilt -23.17 deg
Starting projector calculation for tilt -33.17 deg
Starting projector calculation for tilt -43.17 deg
Starting projector calculation for tilt -53.17 deg
projector calculations finished

Mask 0 Y-shift = 2, Wire direction error = -0.18 deg
Mask 1 Y-shift = 0, Wire direction error = -0.14 deg
Mask 2 Y-shift = 4, Wire direction error = 0.07 deg
Mask 3 Y-shift = 0, Wire direction

D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:368: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Mask 6 Y-shift = 5, Wire direction error = 0.14 deg


D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:1087: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:368: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:368: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, se

Mask 7 Y-shift = 1, Wire direction error = -0.07 deg


D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:1087: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:368: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:368: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, se

Mask 8 Y-shift = 2, Wire direction error = 0.06 deg


D:\Programmes\Git\empyre-patched\empyre\pyramidas\alignment.py:1087: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Average wire direction alignment error: 0.234 deg

trimming all unnecessary empty space from phasemaps

finished alignments


In [128]:
plot_slice=slice(None)
t=[pm.plot_phase() for pm in phasemaps_aligned[plot_slice]]

In [126]:
reconstruction_dimensions

(45, 115, 95)

### 2. Creating 3-D mask

In [186]:
##combine partial information behind the sample (should be done in alignment, this is just a hack)

m=phasemaps_aligned[0].mask.copy()
plt.matshow(m)
free_space=phasemaps_aligned[0].confidence<1
free_space[:,45:]=False
free_space[:95,:]=False

m[free_space]=True
plt.matshow(m)


In [187]:
phasemaps_aligned[0].mask = m

In [188]:
#test if the 3-D mask is created correctly when using a small number of phasemaps

selection=slice(0,1)
z_rotation=0 #deg

no_slices=len(phasemaps_aligned)
z_rots=([z_rotation]*no_slices)
camera_rots=([camera_rotation]*no_slices)
pixel_spacing= phasemaps_aligned[0].a

dimz, dimy, dimx = reconstruction_dimensions
dim=(25,dimy,dimx)


data_test=pa.make_projection_data(phasemaps_aligned[selection], z_rots[selection], tilts[selection], camera_rots[selection], 
                               pixel_spacing, dim=dim, plot_results=False)

data_test.plot_mask(title="3d mask")

Reconstruction voxel number: 273125
Pixel size: 10.2491 nm
3d reconstructions dimensions: (25, 115, 95)
starting projector calculation
1/1; projector calculation finished



In [189]:
#now calculate projectors for all phasemaps
dim=(25,dimy,dimx)
extra_rotation = 0 #deg

x_tilts = tilts + extra_rotation

data_series=pa.make_projection_data(phasemaps_aligned, z_rots, x_tilts, camera_rots, 
                               pixel_spacing, dim=dim, plot_results=False)

data_series.plot_mask(title="combined and smoothed mask")

Reconstruction voxel number: 273125
Pixel size: 10.2491 nm
3d reconstructions dimensions: (25, 115, 95)
starting projector calculation
1/9; 2/9; 3/9; 4/9; 5/9; 6/9; 7/9; 8/9; 9/9; projector calculation finished



#### edit mask manually in MS paint

In [37]:
data_series.set_3d_mask(threshold=1)
fname=pa.save_editable_mask(data_series.mask, axis=1)
print(fname)

png shape (20, 101, 4)
mask_projection.png


In [39]:
data_series.mask = pa.load_png_mask(data_series.mask, "mask_projection - Copy.png", axis=1)
data_series.plot_mask()

.png shape (20, 101, 4)
mask shape (20, 103, 101)


#### refine the mask

In [45]:
#find best 3d mask smoothing
mask_threshold = 0.45
sigma_mask=1
mask3d_0 = data_series.mask.copy()
mask3d = np.where(data_series.mask, 1.0, 0.0)
mask3d = skfl.gaussian(mask3d, sigma = sigma_mask)
mask3d = np.where(mask3d > mask_threshold, True, False)
pu.matshow_n([np.sum(data_series.mask,axis=0), np.sum(mask3d,axis=0)],["old mask","smooth mask"] )
data_series.mask = mask3d
data_series.plot_mask()
data_series.mask = mask3d_0

In [46]:
#save the best smoothed mask
data_series.mask = mask3d


#### Save aligned data series

In [191]:
# with open("data_series.pickle", "wb") as f:
#     pickle.dump(data_series, f)

In [190]:
data_series.plot_phasemaps()

### Quick reconstruction

In [8]:
with open("data_series_unrefined.pickle", "rb") as f:
    data_series=pickle.load(f)
mask0 = data_series.mask.copy()

In [10]:
magdata_rec, cost_fun = pre.reconstruct_from_phasemaps_simple(data_series, verbose=False, max_iter=1000)